In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch as tc
from torch import nn
from torchvision import models
from monai.networks.nets import UNet
from monai.metrics import DiceMetric, HausdorffDistanceMetric

import sys
sys.path.append('/content/drive/MyDrive/Aorta_Segmentation')
from Dataset_loader import Dataset

In [ ]:
def get_losses_from_path(path):
    losses_dict = tc.load(path)
    train_loss_lst = losses_dict["train_loss_lst"]
    val_loss_lst = losses_dict["val_loss_lst"]
    return train_loss_lst, val_loss_lst

Unet_train_losses, Unet_val_losses = get_losses_from_path('/content/drive/MyDrive/Aorta_Segmentation/Losses_values/UNet_losses.pt')
FCN_train_losses, FCN_val_losses = get_losses_from_path('/content/drive/MyDrive/Aorta_Segmentation/Losses_values/FCN_losses.pt')
FCN_pretrained_train_losses, FCN_pretrained_val_losses = get_losses_from_path('/content/drive/MyDrive/Aorta_Segmentation/Losses_values/FCN_pretrained_losses.pt')
Deeplabv3_train_losses, Deeplabv3_val_losses = get_losses_from_path('/content/drive/MyDrive/Aorta_Segmentation/Losses_values/deeplabv3_losses.pt')
Deeplabv3_pretrained_train_losses, Deeplabv3_pretrained_val_losses = get_losses_from_path('/content/drive/MyDrive/Aorta_Segmentation/Losses_values/deeplabv3_pretrained_losses.pt')

titles = [
    "UNet - Losses",
    "FCN Losses",
    "FCN Pretrained Losses",
    "DLv3 Losses",
    "DLv3 Pretrained Losses"
]

train_losses = [
    Unet_train_losses,
    FCN_train_losses,
    FCN_pretrained_train_losses,
    Deeplabv3_train_losses,
    Deeplabv3_pretrained_train_losses
]

val_losses = [
    Unet_val_losses,
    FCN_val_losses,
    FCN_pretrained_val_losses,
    Deeplabv3_val_losses,
    Deeplabv3_pretrained_val_losses
]

plt.figure(figsize=(15, 10), dpi=300)
for i in range(len(train_losses)):
    plt.subplot(2, 3, i + 1)
    plt.plot(train_losses[i], "r-", label="Training Loss")
    plt.plot(val_losses[i], "b-", label="Validation Loss")
    plt.title(titles[i])
    plt.xlabel("Epoch",)
    plt.ylabel("Loss")
    plt.grid(True)
    if i == 0:
        plt.legend()


In [ ]:
with open("/content/drive/MyDrive/Aorta_Segmentation/data_names_split.json", "w") as file:
    data_names_dict = json.load(file)

test_data_names = data_names_dict["test"]
Testing_dataset = Dataset(test_data_names)
Testing_dataset.Preprocess_Data()
Testing_loader = tc.utils.data.DataLoader(Testing_dataset, batch_size=64, shuffle=True)

In [ ]:
def Test_model(model, testing_loader):
    Predictions = []
    Masks_lst = []
    Treshhold = 0.5
    Device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
    model = model.to(Device)
    if tc.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    dice = DiceMetric(include_background=True, reduction="mean", get_not_nans=True)
    hd95 = HausdorffDistanceMetric(include_background=True, reduction="mean", get_not_nans=True)

    model.eval()
    with tc.no_grad():
      for images, masks in testing_loader:
          images = images.to(Device)
          masks = masks.to(Device)
          model = model.to(Device)
          output = model(images.unsqueeze(1))
          if isinstance(output, dict):
                output = output["out"]

          output = tc.nn.Sigmoid()(output)
          output = (output > Treshhold).int()
          Predictions.append(output.cpu())
          Masks_lst.append(masks.unsqueeze(1).cpu())

    Predictions = tc.cat(Predictions, dim=0)
    Masks = tc.cat(Masks_lst, dim=0)

    dice_score = dice(Predictions, Masks)
    hd95_score = hd95(Predictions, Masks)
    dice_score = dice_score[~dice_score.isnan()]
    dice_score = dice_score[~dice_score.isinf()]
    hd95_score = hd95_score[~hd95_score.isnan()]
    hd95_score = hd95_score[~hd95_score.isinf()]
    return dice_score, hd95_score, Predictions, Masks

In [ ]:
model_Unet = UNet(
        spatial_dims = 2,
        in_channels=1,
        out_channels=1,
        channels=[16, 32, 64, 128, 256],
        strides=(2, 2, 2, 2),
        dropout=0.12643763904542435
    )

model_Unet.load_state_dict(tc.load('/content/drive/MyDrive/Aorta_Segmentation/Models/UNet_trained.pth'))
Dice_score_Unet, HD95_score_Unet, Predictions_Unet, Masks_Unet, = Test_model(model_Unet, Testing_loader)
tc.cuda.empty_cache()

print(f"Average Dice score of U-Net: {Dice_score_Unet.mean()}")
print(f"Minimum Dice score of U-Net: {Dice_score_Unet.min()}")
print(f"Maximum Dice score of U-Net: {Dice_score_Unet.max()}")
print(f"Std Dice score of U-Net: {Dice_score_Unet.std()}")
print("---------------")
print(f"Average HD95 score of U-Net: {HD95_score_Unet.mean()}")
print(f"Minimum HD95 score of U-Net: {HD95_score_Unet.min()}")
print(f"Maximum HD95 score of U-Net: {HD95_score_Unet.max()}")
print(f"Std HD95 score of U-Net: {HD95_score_Unet.std()}")

In [ ]:
model_FCN = models.segmentation.fcn_resnet50(weights=None, num_classes=1)
model_FCN.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

model_FCN.load_state_dict(tc.load('/content/drive/MyDrive/Aorta_Segmentation/Models/FCN_trained.pth'))
Dice_score_FCN, HD95_score_FCN, Predictions_FCN, Masks_FCN, = Test_model(model_FCN, Testing_loader)
tc.cuda.empty_cache()

print(f"Average Dice score of FCN: {Dice_score_FCN.mean()}")
print(f"Minimum Dice score of FCN: {Dice_score_FCN.min()}")
print(f"Maximum Dice score of FCN: {Dice_score_FCN.max()}")
print(f"Std Dice score of FCN: {Dice_score_FCN.std()}")
print("---------------")
print(f"Average HD95 score of FCN: {HD95_score_FCN.mean()}")
print(f"Minimum HD95 score of FCN: {HD95_score_FCN.min()}")
print(f"Maximum HD95 score of FCN: {HD95_score_FCN.max()}")
print(f"Std HD95 score of FCN: {HD95_score_FCN.std()}")

In [ ]:
model_FCN_pretrained = models.segmentation.fcn_resnet50(weights=FCN_ResNet50_Weights)
model_FCN_pretrained.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_FCN_pretrained.classifier[4] = nn.Conv2d(512, 1, kernel_size=1)

model_FCN_pretrained.load_state_dict(tc.load('/content/drive/MyDrive/Aorta_Segmentation/Models/FCN_pretrained_trained.pth'))
Dice_score_FCN_pretrained, HD95_score_FCN_pretrained, Predictions_FCN_pretrained, Masks_FCN_pretrained, = Test_model(model_FCN_pretrained, Testing_loader)
tc.cuda.empty_cache()

print(f"Average Dice score of FCN pretrained: {Dice_score_FCN_pretrained.mean()}")
print(f"Minimum Dice score of FCN pretrained: {Dice_score_FCN_pretrained.min()}")
print(f"Maximum Dice score of FCN pretrained: {Dice_score_FCN_pretrained.max()}")
print(f"Std Dice score of FCN pretrained: {Dice_score_FCN_pretrained.std()}")
print("---------------")
print(f"Average HD95 score of FCN pretrained: {HD95_score_FCN_pretrained.mean()}")
print(f"Minimum HD95 score of FCN pretrained: {HD95_score_FCN_pretrained.min()}")
print(f"Maximum HD95 score of FCN pretrained: {HD95_score_FCN_pretrained.max()}")
print(f"Std HD95 score of FCN pretrained: {HD95_score_FCN_pretrained.std()}")

In [ ]:
model_deeplabv3 = models.segmentation.deeplabv3_mobilenet_v3_large(weights=None, num_classes=1)
model_deeplabv3.backbone._modules["0"]._modules["0"] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False)

model_deeplabv3.load_state_dict(tc.load('/content/drive/MyDrive/Aorta_Segmentation/Models/deeplabv3_trained.pth'))
Dice_score_deeplabv3, HD95_score_deeplabv3, Predictions_deeplabv3, Masks_deeplabv3, = Test_model(model_deeplabv3, Testing_loader)
tc.cuda.empty_cache()

print(f"Average Dice score of deeplabv3: {Dice_score_deeplabv3.mean()}")
print(f"Minimum Dice score of deeplabv3: {Dice_score_deeplabv3.min()}")
print(f"Maximum Dice score of deeplabv3: {Dice_score_deeplabv3.max()}")
print(f"Std Dice score of deeplabv3: {Dice_score_deeplabv3.std()}")
print("---------------")
print(f"Average HD95 score of deeplabv3: {HD95_score_deeplabv3.mean()}")
print(f"Minimum HD95 score of deeplabv3: {HD95_score_deeplabv3.min()}")
print(f"Maximum HD95 score of deeplabv3: {HD95_score_deeplabv3.max()}")
print(f"Std HD95 score of deeplabv3: {HD95_score_deeplabv3.std()}")

In [ ]:
model_deeplabv3_pretrained = models.segmentation.deeplabv3_mobilenet_v3_large(weights=DeepLabV3_MobileNet_V3_Large_Weights)
model_deeplabv3_pretrained.backbone._modules["0"]._modules["0"] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False)
model_deeplabv3_pretrained.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)

model_deeplabv3_pretrained.load_state_dict(tc.load('/content/drive/MyDrive/Aorta_Segmentation/Models/deeplabv3_pretrained_trained.pth'))
Dice_score_deeplabv3_pretrained, HD95_score_deeplabv3_pretrained, Predictions_deeplabv3_pretrained, Masks_deeplabv3_pretrained, = Test_model(model_deeplabv3_pretrained, Testing_loader)
tc.cuda.empty_cache()

print(f"Average Dice score of deeplabv3_pretrained: {Dice_score_deeplabv3_pretrained.mean()}")
print(f"Minimum Dice score of deeplabv3_pretrained: {Dice_score_deeplabv3_pretrained.min()}")
print(f"Maximum Dice score of deeplabv3_pretrained: {Dice_score_deeplabv3_pretrained.max()}")
print(f"Std Dice score of deeplabv3_pretrained: {Dice_score_deeplabv3_pretrained.std()}")
print("---------------")
print(f"Average HD95 score of deeplabv3_pretrained: {HD95_score_deeplabv3_pretrained.mean()}")
print(f"Minimum HD95 score of deeplabv3_pretrained: {HD95_score_deeplabv3_pretrained.min()}")
print(f"Maximum HD95 score of deeplabv3_pretrained: {HD95_score_deeplabv3_pretrained.max()}")
print(f"Std HD95 score of deeplabv3_pretrained: {HD95_score_deeplabv3_pretrained.std()}")

In [ ]:
Model_names = ["Unet",
               "FCN",
               "FCN_pretrained",
               "deeplabv3",
               "deeplabv3_pretrained"]

Dices_lst = [Dice_score_Unet.mean().item(),
             Dice_score_FCN.mean().item(),
             Dice_score_FCN_pretrained.mean().item(),
             Dice_score_deeplabv3.mean().item(),
             Dice_score_deeplabv3_pretrained.mean().item()]


plt.figure(figsize=(10,5))
ax = sns.barplot(x=Model_names, y=Dices_lst, palette=sns.color_palette(palette='cool'))
for i, value in enumerate(Dices_lst):
    ax.text(i, value + 0.02, f'{value:.2f}', ha='center', va='bottom', fontsize=12)
plt.title("Dice Comparison")
plt.ylabel("Average Dice score")
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
HD95_lst = [HD95_score_Unet.mean().item(),
            HD95_score_FCN.mean().item(),
            HD95_score_FCN_pretrained.mean().item(),
            HD95_score_deeplabv3.mean().item(),
            HD95_score_deeplabv3_pretrained.mean().item()]

plt.figure(figsize=(10,5))
ax_2 = sns.barplot(x=Model_names, y=HD95_lst, palette=sns.color_palette(palette='cool'))
for i, value in enumerate(HD95_lst):
    ax_2.text(i, value + 0.02, f'{value:.2f}', ha='center', va='bottom', fontsize=12)
plt.title("HD95 Comparison")
plt.ylabel("Average HD95 score")
plt.ylim(0, 23)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()